# Time Series Analysis

## Step 1 of 3: Reading Credit Card Transaction Data

In [12]:
import pandas as pd
import plotext as plt
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error
import numpy as np

In [13]:
df = pd.read_csv('assets/fast_food_credit_card_transaction_data.csv')
df.head()

,Unnamed: 0,transaction_id,transaction_date,cardholder_name,card_number,card_type,merchant_name,merchant_category,merchant_state,merchant_city,transaction_status,transaction_amount,merchant_category_code,fraud_flag
0,0,5WFUH5-VMPB-RHOXKV,2020-01-17 16:30:00,Miss Vanessa Briggs MD,4533948622139044,visa,McDonald's,Fast Food,Montana,Missoula,Successful,11.516379,MCC 5814,0
1,1,TW1LD6-3CGZ-MQW4KN,2022-08-26 22:41:00,Casey Lyons,4350240875308199,visa,Domino's Pizza,Fast Food,Ohio,Cleveland,Pending,12.739792,MCC 5814,0
2,2,DKKV7Q-H2LA-406TWY,2018-04-18 20:25:00,Cynthia Munoz,4756687869818916,visa,McDonald's,Fast Food,Massachusetts,Worcester,Declined,20.899888,MCC 5814,0
3,3,3D0M9F-IAZ0-UANAA7,2018-03-10 12:07:00,Jeffrey Atkinson,4806484616398735,visa,McDonald's,Fast Food,Ohio,Columbus,Pending,16.466555,MCC 5814,0
4,4,DYAGZ9-7S25-N1FXOQ,2018-09-15 06:20:00,Heather Walker,4773028361743188,visa,Five Guys,Fast Food,Alaska,Anchorage,Declined,3.646512,MCC 5814,0


In [14]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

In [15]:
fast_food = df[df['merchant_name'] == "McDonald's"]
fast_food.head()

,Unnamed: 0,transaction_id,transaction_date,cardholder_name,card_number,card_type,merchant_name,merchant_category,merchant_state,merchant_city,transaction_status,transaction_amount,merchant_category_code,fraud_flag
0,0,5WFUH5-VMPB-RHOXKV,2020-01-17 16:30:00,Miss Vanessa Briggs MD,4533948622139044,visa,McDonald's,Fast Food,Montana,Missoula,Successful,11.516379,MCC 5814,0
2,2,DKKV7Q-H2LA-406TWY,2018-04-18 20:25:00,Cynthia Munoz,4756687869818916,visa,McDonald's,Fast Food,Massachusetts,Worcester,Declined,20.899888,MCC 5814,0
3,3,3D0M9F-IAZ0-UANAA7,2018-03-10 12:07:00,Jeffrey Atkinson,4806484616398735,visa,McDonald's,Fast Food,Ohio,Columbus,Pending,16.466555,MCC 5814,0
11,12,4YOZP4-ZEQR-7X8B4P,2019-02-09 21:50:00,Larry Wright,4184070850475263,visa,McDonald's,Fast Food,New York,Buffalo,Successful,20.123632,MCC 5814,0
13,14,49WVGA-PO5T-ROA5W6,2022-12-25 16:11:00,Tyler Barnes,4780834378780084,visa,McDonald's,Fast Food,Delaware,Wilmington,Pending,8.026983,MCC 5814,0


In [16]:
plt.clear_figure()

plt.hist(fast_food['transaction_amount'], bins=40)
plt.show()

     ┌─────────────────────────────────────────────────────────────────────────┐
229.0┤██                                                                       │
     │██  ██                                                                   │
     │██  ██                                                                   │
190.8┤██  ████                                                                 │
     │██  ████                                                                 │
     │████████                                                                 │
152.7┤████████                                                                 │
     │████████                                                                 │
     │██████████                                                               │
114.5┤██████████                                                               │
     │███████████                                                              │
     │█████████████         

In [17]:
z_scores = np.abs((fast_food['transaction_amount'] - fast_food['transaction_amount'].mean()) / fast_food['transaction_amount'].std())
z_scores

0       0.216769
2       0.517897
3       0.170797
11      0.457121
13      0.489965
          ...   
8104    0.507775
8108    0.790466
8109    0.184604
8112    2.507452
8117    0.000204
Name: transaction_amount, Length: 1660, dtype: float64

In [18]:
threshold = 3

In [19]:
fast_food = fast_food[z_scores < threshold]

In [20]:
plt.clear_figure()
plt.hist(fast_food['transaction_amount'], bins=40)
plt.show()

   ┌───────────────────────────────────────────────────────────────────────────┐
138┤██                                                                         │
   │██                                                                         │
   │██     ███                                                                 │
115┤██     █████                                                               │
   │████   ███████                                                             │
   │██████ ███████                                                             │
 92┤██████████████                                                             │
   │████████████████                                                           │
   │████████████████                                                           │
 69┤████████████████ ██                                                        │
   │███████████████████                                                        │
   │████████████████████████

In [21]:
df_grouped = fast_food.groupby(fast_food['transaction_date'].dt.date)['transaction_amount'].sum().reset_index()
df_grouped

,transaction_date,transaction_amount
0,2018-01-01,2.542163
1,2018-01-02,68.364142
2,2018-01-03,3.554909
3,2018-01-04,36.432596
4,2018-01-05,8.548116
...,...,...
1156,2023-08-26,7.405734
1157,2023-08-27,68.079695
1158,2023-08-28,14.356225
1159,2023-08-29,2.012224


In [22]:
plt.clear_figure()
merchants = ["Subway", "Domino's Pizza"]
for group in merchants:
    subgroup = df[df['merchant_name'] == group].copy()
    merchant_category = list(set(subgroup['merchant_category']))[0]
    z_scores = np.abs((subgroup['transaction_amount'] - subgroup['transaction_amount'].mean()) / subgroup['transaction_amount'].std())
    threshold = 3
    subgroup = subgroup[z_scores < threshold]
    subgroup['year'] = subgroup['transaction_date'].dt.year
    subgroup['year'] = subgroup['year'].astype(str)
    subgroup['year'] =  "01" + "/" + "01"  + "/" + subgroup['year'] 
    df_grouped = subgroup.groupby('year')['transaction_amount'].sum().reset_index()
    df_grouped = df_grouped.set_index('year').sort_index()
    plt.plot(df_grouped.index, df_grouped['transaction_amount'], label=f'{merchant_category}: {group} Total Transaction Amount by year')
    plt.title(f'{merchant_category}: {group} Total Transaction Amount by year')
    plt.xlabel('Year')
    plt.ylabel('Transaction Amount')
    plt.grid()
    plt.show()


                  Fast Food: Subway Total Transaction Amount by year            
      ┌────────────────────────────────────────────────────────────────────────┐
4414.7┤ ▞▞ Fast Food: Subway Total Transaction Amount by year                  │
      │▄▄▄▄▄▀▀▀▀▀            ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▝▀▀▀▀▀▀▀▀▀▀▀▀▀▌              │
      │                                                         ▝▖             │
4082.8┤                                                          ▝▖            │
      │                                                           ▝▖           │
3750.9┤                                                            ▝▖          │
      │                                                             ▝▖         │
      │                                                              ▝▖        │
3419.0┤                                                               ▝▖       │
      │                                                                ▚       │
      │                     

## Step 2 of 3: Data Preparation for Time Series Forecasting

In [23]:
df_dominos = df[df['merchant_name'] == "Domino's Pizza"]
df_dominos.head()

,Unnamed: 0,transaction_id,transaction_date,cardholder_name,card_number,card_type,merchant_name,merchant_category,merchant_state,merchant_city,transaction_status,transaction_amount,merchant_category_code,fraud_flag
1,1,TW1LD6-3CGZ-MQW4KN,2022-08-26 22:41:00,Casey Lyons,4350240875308199,visa,Domino's Pizza,Fast Food,Ohio,Cleveland,Pending,12.739792,MCC 5814,0
20,22,0UYDHD-VKR9-2T904X,2021-12-31 15:32:00,Denise Goodwin,4339361377179312,visa,Domino's Pizza,Fast Food,New Hampshire,Nashua,Successful,13.507401,MCC 5814,0
23,25,26SECF-WY0Y-OS5Z9A,2018-11-04 19:45:00,Jason Vaughan Jr.,4547442055365577,visa,Domino's Pizza,Fast Food,Illinois,Chicago,Pending,20.207456,MCC 5814,1
30,32,XPFO1U-SBE3-RYNK2A,2021-10-12 21:48:00,Miranda Bailey,4763802930709009,visa,Domino's Pizza,Fast Food,Delaware,Dover,Successful,2.302596,MCC 5814,1
35,37,QM9C9Y-J5ID-TY430V,2022-11-15 22:53:00,Stephen Smith,4515961171701173,visa,Domino's Pizza,Fast Food,Ohio,Cleveland,Pending,14.416764,MCC 5814,0


In [24]:
z_scores = np.abs((df_dominos['transaction_amount'] - df_dominos['transaction_amount'].mean()) / df_dominos['transaction_amount'].std())
z_scores

1       0.169895
20      0.112268
23      0.390728
30      0.953450
35      0.043999
          ...   
8099    0.406298
8100    0.770676
8101    0.803007
8115    0.914489
8124    0.300139
Name: transaction_amount, Length: 1568, dtype: float64

In [25]:
threshold = 3

In [26]:
df_dominos = df_dominos[z_scores < threshold]

In [27]:
df_grouped.index = pd.to_datetime(df_grouped.index)
df_grouped.head()

,transaction_amount
year,
2018-01-01,3868.674632
2019-01-01,3903.864402
2020-01-01,4436.718785
2021-01-01,3936.216946
2022-01-01,3933.414038


In [30]:
# the code

cutoff_point1 = pd.to_datetime('2021-06-01')
cutoff_point2 = pd.to_datetime('2023-06-01')

In [41]:
train_data = df_grouped[df_grouped.index < cutoff_point1]

train_data['year'] = train_data.index

train_data['year'] = train_data['year'].dt.year

train_data['year'] = train_data['year'].astype(str)

train_data['year'] =  "01" + "/" + "01"  + "/" + train_data['year']

train_data = train_data.set_index('year').sort_index()

train_data

/tmp/ipykernel_64112/2150860022.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['year'] = train_data.index
/tmp/ipykernel_64112/2150860022.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['year'] = train_data['year'].dt.year
/tmp/ipykernel_64112/2150860022.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,transaction_amount
year,
01/01/2018,3868.674632
01/01/2019,3903.864402
01/01/2020,4436.718785
01/01/2021,3936.216946


In [44]:
test_data = df_grouped[(df_grouped.index >= cutoff_point1) & (df_grouped.index < cutoff_point2)]
test_data['year'] = test_data.index
test_data['year'] = test_data['year'].dt.year 
test_data['year'] = test_data['year'].astype(str)
test_data['year'] =  "01" + "/" + "01"  + "/" + test_data['year']
test_data = test_data.set_index('year').sort_index()

test_data

/tmp/ipykernel_64112/3963236384.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['year'] = test_data.index
/tmp/ipykernel_64112/3963236384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['year'] = test_data['year'].dt.year
/tmp/ipykernel_64112/3963236384.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

,transaction_amount
year,
01/01/2022,3933.414038
01/01/2023,2931.009929


## Step 3 of 3: Train the AutoARIMA Model on the Train Data

In [45]:
model = auto_arima(train_data['transaction_amount'])
model

ARIMA(order=(0, 0, 0), scoring_args={}, suppress_warnings=True)

In [46]:
predictions = model.predict(n_periods=len(test_data))

In [48]:
mse = mean_squared_error(test_data['transaction_amount'], predictions)
mse

616208.8268479431

In [49]:
rmse = mse ** 0.5

In [50]:
print(f'RMSE: {rmse}')

RMSE: 784.989698561671


In [51]:
plt.clear_data()

In [52]:
plt.plot(train_data.index, train_data['transaction_amount'], label='Training')

In [53]:
plt.plot(test_data.index, test_data['transaction_amount'], label='Testing')

In [54]:
# next plot predictions
plt.plot(test_data.index, predictions, label='Predictions')

In [55]:
plt.title('Transaction Amount Analysis')

In [56]:
plt.xlabel('Index')

In [57]:
plt.ylabel('Transaction Amount')

In [58]:
plt.grid()

In [59]:
plt.show()

                              Transaction Amount Analysis                       
      ┌────────────────────────────────────────────────────────────────────────┐
4436.7┤ ▞▞ Training               ▗▞▄                                          │
      │ ▞▞ Testing              ▄▞▘  ▀▚▖                                       │
      │ ▞▞ Predictions        ▄▀       ▝▀▄▖                                    │
4185.8┤                    ▗▞▀            ▝▚▄                                  │
      │                  ▄▞▘                 ▀▚▖                ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│
3934.8┤                ▄▀                      ▝▀▄▖             ▖              │
      │▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀                                         ▝▖             │
      │                                                          ▝▄            │
3683.9┤                                                            ▚           │
      │                                                             ▀▖         │
      │                     